<a href="https://colab.research.google.com/github/BotenAqua/WarframeInventoryHelper/blob/master/WarframeInventoryHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests

In [70]:
def pageTableExtractor(page, first_row_is_header = True):
  try:
    data = requests.get(page)
  except NameError:
    import requests
    data = requests.get(page)
  except ModuleNotFoundError:
    print("Requests module not found! Aborting...")
    return
  data = data.text
  #print(data.find("<table"), data.find("</table>"))
  data = data[data.find("<table"):data.find("</table>")]

  if first_row_is_header:
    header = []
    cut = data.find("</tr>")
    row = data[data.find("<tr>")+4:cut]
    data = data[cut+5:]
    del cut
    for _ in range(row.count("</th>")):
      do = row[3:].find("<")+2
      #print(row, row.find(">")+1,do)
      header.append(row[row.find(">")+1:do])
      row = row[do+6:]
    #print(header)
    content = []
    for _ in range(data.count("</tr>")):
      do = data.find("</tr>")+5
      #print(data.find("<tr"),do)
      content.append(data[data.find("<tr"):do])
      data = data[do:]
    print(content[-1])
    #oczyscic dane i zwrocic jako header + body

  return

pageTableExtractor("https://warframe.fandom.com/wiki/Void_Relic/ByRewards/SimpleTable")


<tr>
<td><a href="/wiki/Zhuge_Prime" title="Zhuge Prime">Zhuge Prime</a></td><td>String</td><td>Axi</td><td><span style="border-bottom: 1px dotted;" class="relic-tooltip" data-param="Axi M1"><a href="/wiki/Axi_M1" title="Axi M1">M1</a></span></td><td>Common</td><td>No
</td></tr>
